In [210]:
from utils import *


row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
diag_1=[]
for i in range(0,len(row_units)):
    for j in range(0,len(column_units)):
        if(i==j):
            diag_1.append(row_units[i][j])
diag_2=[]
for i in range(0,len(row_units)):
    #print("i= "+str(i));
    for j in range(len(column_units)-1,-1,-1):
        #print("j= "+str(j));
        if(i+j==8):
            #print("j inside= "+str(j))
            diag_2.append(row_units[i][j])

    
unitlist = row_units + column_units + square_units + [diag_1] + [diag_2]

# TODO: Update the unit list to add the new diagonal units
unitlist = unitlist
# Must be called after all units (including diagonals) are added to the unitlist
units = extract_units(unitlist, boxes)
peers = extract_peers(units, boxes)


In [211]:
#Harder sodocu
#grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
#grid='..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
#d_g='2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
#new_value2=(grid2values(d_g))

In [212]:
#new_value2=eliminate(new_value2)
#new_value2=only_choice(new_value2)
#display(new_value2)
#reduce_puzzle(new_value2)

In [221]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.

    The naked twins strategy says that if you have two or more unallocated boxes
    in a unit and there are only two digits that can go in those two boxes, then
    those two digits can be eliminated from the possible assignments of all other
    boxes in the same unit.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the naked twins eliminated from peers

    Notes
    -----
    Your solution can either process all pairs of naked twins from the input once,
    or it can continue processing pairs of naked twins until there are no such
    pairs remaining -- the project assistant test suite will accept either
    convention. However, it will not accept code that does not process all pairs
    of naked twins from the original input. (For example, if you start processing
    pairs of twins and eliminate another pair of twins before the second pair
    is processed then your code will fail the PA test suite.)

    The first convention is preferred for consistency with the other strategies,
    and because it is simpler (since the reduce_puzzle function already calls this
    strategy repeatedly).
    """
    # TODO: Implement this function!
    #Get all keys that has length has 2
    box_2=[sqr for sqr in boxes if len(values[sqr])==2]
    #display(values)
    #get all the twins paird
    list_twins=[]
    for sqr in box_2:
        #print(sqr)
        for y in peers[sqr]:
            if(values[sqr]==values[y]):
                if(sqr!=y):
                    list_twins.append((sqr,y))
    #print(list_twins)        
    
    common_dict=dict()
    for(one, two) in list_twins:
        common_dict[one]=list(peers[one]-(peers[one]-peers[two]))

    for box, box_new in common_dict.items():
        #print(box)
        #print(box_new)
        for singlebox in box_new:
            digit=values[box]
            for dig in digit:
                values[singlebox]=values[singlebox].replace(dig,'')
    
    return values
    raise NotImplementedError

In [222]:
def eliminate(values):
   
    n_values = (u for u in values.keys() if len(values[u])==1)
    
    for keys in n_values:
        #print(keys)
        n=values[keys]
        #print(n)
        for peer in peers[keys]:
            #print(peer)
            values[peer]=values[peer].replace(n,'')          
    return values

In [223]:
def only_choice(values):
    for unit in unitlist:
        #print(unit)
        for digit in '123456789':
            #print(digit)
            #display(values)
            dplaces = [box for box in unit if digit in values[box]]
            #print(dplaces)
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
            #display(values)
    return values

In [224]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)
        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)
        values = naked_twins(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values


In [228]:
def search(values):
    """Apply depth first search to solve Sudoku puzzles in order to solve puzzles
    that cannot be solved by repeated reduction alone.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict or False
        The values dictionary with all boxes assigned or False

    Notes
    -----
    You should be able to complete this function by copying your code from the classroom
    and extending it to call the naked twins strategy.
    """
    # TODO: Copy your code from the classroom to complete this function
    
    values = reduce_puzzle(values)

    
    if values is False:
        return False ## Failed earlier
    if all(len(values[sqr]) == 1 for sqr in boxes): 
        return values ## Solved!
        
        
        
    
    # Choose one of the unfilled squares with the fewest possibilities

    val,sqr = min((len(values[sqr]), sqr) for sqr in boxes if len(values[sqr]) > 1)
    #print('values')
    #print(val)
    #print(sqr)

    # Now use recursion to solve each one of the resulting sudokus, and if one returns a value (not False), return that answer!
    for value in values[sqr]:
        next_state = values.copy()
        next_state[sqr] = value
        
        board = search(next_state)
        
        if board:
            return board
        
    raise NotImplementedError

In [229]:
def solve(grid):
    """Find the solution to a Sudoku puzzle using search and constraint propagation

    Parameters
    ----------
    grid(string)
        a string representing a sudoku grid.
        
        Ex. '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'

    Returns
    -------
    dict or False
        The dictionary representation of the final sudoku grid or False if no solution exists.
    """
    values = grid2values(grid)
    values = search(values)
    return values

In [230]:
if __name__ == "__main__":
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    display(grid2values(diag_sudoku_grid))
    result = solve(diag_sudoku_grid)
    display(result)

    try:
        import PySudoku
        PySudoku.play(grid2values(diag_sudoku_grid), result, history)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')

    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678